# Оценивание
За выполнени каждого задания 1 балл

4 балла -> отл

3 балла -> хор

И тд

### Задание -1 (вопрос к первому уроку)
Можно ли использовать MSE в каком-либо виде

MSE — чаще используют как меру качества внутри самой модели, когда оценивается рейтинг по непрерывной шкале (0-10)  (градиентные спуски итп, направленные на уменьшение значения), хотя если я правильно нагуглила - сейчас более популярны решения с RMSE, так как они дают лучший результат. В случае использования  MSE модель обучается только на части доступных взаимодействий и тестируется на оставшихся.

Проблема MSE в неинтерпретируемости

### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - что это за товар?  
Зачем он нужен?  
Используя этот товар мы смещаем качество рекомендаций.
В какую сторону?   
Можно ли удалить этот товар?   
Уберите этот товар и сравните с качеством на семинаре.

- товар 999999 отвечает за редкие товары, не попавшие в топ - 5000
- он нужен для коррекции вектора, чтобы модель могла учитывать остальные значения как одно, а не просто их выкидавать из датасета
- в какую сторону? добавляет шум вероятно?
- этот товар можно удалить, но модель потеряет информативность и перестанет учитывать остальные товары (грубо говоря, пользователь, который купил 1 товар из топ-5000 и 10 из непопулярных, станет похож на пользователя, который купил только 2 товара из топ-5000 и 1 совпадет)

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random
import math

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [20]:
data = pd.read_csv('./retail_train.csv')
result = pd.read_csv('./predictions_basic.csv')

test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]

In [22]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [23]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = None
data_train = data_train.dropna()

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

/Users/ekaterinarazdumina/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


item_id,202291.0,397896.0,420647.0,480014.0,545926.0,707683.0,731106.0,818980.0,819063.0,819227.0,...,15778533.0,15831255.0,15926712.0,15926775.0,15926844.0,15926886.0,15927403.0,15927661.0,15927850.0,16809471.0
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
np.sort(data.item_id.unique())

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [25]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)


CPU times: user 1.7 s, sys: 48 ms, total: 1.75 s
Wall time: 1.75 s


In [26]:
%%time

result['itemitem_no_9999'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

KeyError: 650

- не понимаю что делаю не так - намудрила с выбрасыванием? Почему то когда замеряла качество на p@5 у модели вебинара (загоняла actual и itemitem) получила 0
- все время ловлю эту ошибку 650 (( 

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. вес = log(sales_sum товара)
- Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

In [36]:
def weighted_random_recommendation(items, weights, n=5):
    

    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    items = list(items)
    weights = list(weight)
    weights_log = []
    
    for w in weights:
        weights.append(math.log(w))
    
    recs = np.random.choices(items, weights_log, k=n) 
    
    return recs.tolist()

Сделайте предсказания

In [37]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()
weight = popularity['sales_value']

#хотела через apply math.log разу сделать логарифмы, но почему то вываливалась ошибка про сам модуль math
# (та же ошибка что вывалилась ниже)

In [38]:
%%time

items = data_train.item_id.unique()

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items, weight, n=5))
result.head(2)

ValueError: math domain error

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество? Почему?

In [10]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[164623, 899423, 1945197, 1097044, 194998]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[539998, 12810340, 847381, 1230635, 884316]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [11]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    print(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [39]:
?map

In [41]:
result['actual'].map(lambda x: x[1:-1].split(', ')).apply(lambda x: list(map(int, x)))[0]

[821867,
 834484,
 856942,
 865456,
 889248,
 907957,
 914190,
 943316,
 951954,
 954486,
 958046,
 962568,
 969231,
 971585,
 979707,
 986947,
 990656,
 995242,
 1004906,
 1005186,
 1042083,
 1050310,
 1060819,
 1062002,
 1064441,
 1069733,
 1074612,
 1082185,
 1131115,
 1132771,
 6534544,
 13876341,
 15971874,
 17178953,
 883616,
 917704,
 931860,
 961554,
 1002032,
 1031190,
 8090541,
 8293439,
 9297615,
 9527329,
 15926712,
 1049998,
 861272,
 869465,
 877373,
 908213,
 933913,
 940947,
 945809,
 959316,
 978974,
 1031697,
 1041796,
 1048918,
 1081189,
 1101422,
 1115576,
 1122428,
 1132231,
 1132814,
 5577022,
 8091601,
 9296986,
 9677939,
 10356149,
 13417048,
 15741823,
 15830875]

In [42]:
result.apply(lambda x: precision_at_k(x['popular_recommendation'], x['actual'],  5), axis=1)

AttributeError: 'str' object has no attribute 'tocsr'

### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
- Попробуйте стратегии ансамблирования изученных алгоритмов


In [ ]:
# your_code

### Задание 4. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.